# En-Fr Translation
Generate translations on MuST-SHE dataset with WMT14 En-Fr Transformer model

In [4]:
import torch

In [2]:
%cd /export/data4/vzhekova/MuST-SHE_v1.2/MuST-SHE-v1.2-data/tsv

!cut -f5 MONOLINGUAL.fr_v1.2.tsv > data.en-fr.en
!cut -f6 MONOLINGUAL.fr_v1.2.tsv > data.en-fr.fr

/export/data4/vzhekova/MuST-SHE_v1.2/MuST-SHE-v1.2-data/tsv


In [3]:
PATH="/export/data4/vzhekova/biases-data/En-Fr"

%cd $PATH

/export/data4/vzhekova/biases-data/En-Fr


In [249]:
!echo -e "\nFirst lines of English:\n"
!head data.en-fr.en
!echo -e "\nFirst lines of French:\n"
!head data.en-fr.fr


First lines of English:

﻿Now, I thought, "How could I really capture this?
I mean, from this entry, it would seem that I was born into a world that perceived someone like me to have nothing positive whatsoever going for them, when in fact, today I'm celebrated for the opportunities and adventures my life has procured.
So, I immediately went to look up the 2009 online edition, expecting to find a revision worth noting.
His name was Dr. Pizzutillo, an Italian American, whose name, apparently, was too difficult for most Americans to pronounce, so he went by Dr. P. And Dr. P always wore really colorful bow ties and had the very perfect disposition to work with children.
And, one day, he came in to my session — exhaustive and unforgiving, these sessions — and he said to me, "Wow.
Now, of course, this was a simple ploy on Dr. P's part to get me to do the exercises I didn't want to do before the prospect of being the richest five-year-old in the second floor ward, but what he effectively di

# Data preprocessing

- Tokenization

In [4]:
# Tokenize text; MuST-SHE dataset
# TODO: write script for preprocessing
from sacremoses import MosesPunctNormalizer
from sacremoses import MosesTokenizer, MosesDetokenizer
from __future__ import print_function

mpn = MosesPunctNormalizer()
mt_en = MosesTokenizer(lang='en')
md_en = MosesDetokenizer(lang='en')

with open('data.en-fr.en') as fin, open('data.en-fr.tok.en','w') as fout:
    for line in fin:
        tokens = mt_en.tokenize(mpn.normalize(line), return_str=True)
        print(tokens, end='\n', file=fout) 
        
mt_fr = MosesTokenizer(lang='fr')
md_fr = MosesDetokenizer(lang='fr')

with open('data.en-fr.fr') as fin, open('data.en-fr.tok.fr','w') as fout:
    for line in fin:
        tokens = mt_fr.tokenize(mpn.normalize(line), return_str=True)
        print(tokens, end='\n', file=fout)

print('Finished tokenizing.')

Finished tokenizing.


In [140]:
# Detokenize text

# mpn = MosesPunctNormalizer()
# mt = MosesTokenizer(lang='en')
# md = MosesDetokenizer(lang='en')

# md.detokenize(mt.tokenize(md.detokenize(mpn.normalize('So, I immediately went to look up the 2009 online edition, expecting to find a revision worth noting.').split())))

# with open('data.en-fr.tok.en') as fin, open('data.en-fr.detok.en','w') as fout:
#     for line in fin:
#         tokens = md_en.detokenize(line.split(), return_str=True)
#         print(tokens, end='\n', file=fout) 

# with open('data.en-fr.tok.fr') as fin, open('data.en-fr.detok.fr','w') as fout:
#     for line in fin:
#         tokens = md_fr.detokenize(line.split(), return_str=True)
#         print(tokens, end='\n', file=fout)

# print('Finished detokenizing.')

Finished detokenizing.


- Subword tokenization

In [5]:
# Dividing tokenized text into subword units

!subword-nmt learn-bpe -s 32000 < data.en-fr.tok.en > sw.model.en
!subword-nmt apply-bpe -c sw.model.en < data.en-fr.tok.en > sw.data.en

!subword-nmt learn-bpe -s 32000 < data.en-fr.tok.fr > sw.model.fr
!subword-nmt apply-bpe -c sw.model.fr < data.en-fr.tok.fr > sw.data.fr

print('Finished subword.')

 16%|#####7                               | 5014/32000 [00:10<00:57, 473.37it/s]
Finished subword.


In [7]:
!echo -e "\nFirst lines of tokenized English:\n"
!head sw.data.en-fr.en

!echo -e "\nFirst lines of tokenized French:\n"
!head sw.data.en-fr.fr


First lines of tokenized English:

Now , I thought , &quot; How could I really capture this ?
I mean , from this ent@@ ry , it would seem that I was born into a world that percei@@ ved someone like me to have nothing positive whatsoever going for them , when in fact , today I &apos;m celebr@@ ated for the oppor@@ t@@ uni@@ ties and adventures my life has pro@@ cur@@ ed .
So , I immediately went to look up the 200@@ 9 online e@@ dition , expec@@ ting to find a re@@ vision wor@@ th n@@ oting .
His name was Dr. P@@ i@@ zz@@ u@@ til@@ lo , an Italian American , whose name , apparently , was too difficult for most Americ@@ ans to pr@@ on@@ ounce , so he went by Dr. P@@ . And Dr. P always wor@@ e really colorful bow ties and had the very perfect disp@@ os@@ i@@ tion to work with children .
And , one day , he came in to my session - exha@@ us@@ tive and un@@ for@@ giving , these sessions - and he said to me , &quot; W@@ ow .
Now , of course , this was a simple p@@ loy on Dr. P &apos;s part t

- Binarize data

In [9]:
# Binarize the data for training; test with moses tokenizer and subword_nmt failed -> BLEU score of 4

# reuse model dict; solution for dictionary size to match test dataset
# map words appearing less than threshold times to unknown 
# --tokenizer moses \
# --bpe subword_nmt \
!fairseq-preprocess \
    --source-lang en \
    --target-lang fr \
    --testpref sw.data.en-fr \
    --srcdict wmt14.en-fr.joined-dict.transformer/dict.en.txt \
    --tgtdict wmt14.en-fr.joined-dict.transformer/dict.fr.txt \
    --destdir data-bin \
    --thresholdtgt 0 \
    --thresholdsrc 0 \
    --workers 8

2023-02-23 15:57:58 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, source_lang='en', srcdict='wmt14.en-

# Translation

In [10]:
%cd $PATH

/export/data4/vzhekova/biases-data/En-Fr


- Beam search

In [11]:
# Generate translations
!fairseq-generate data-bin  \
    --task translation \
    --source-lang en \
    --target-lang fr \
    --path wmt14.en-fr.joined-dict.transformer/model.pt \
    --beam 5 \
    --batch-size 256 \
    --memory-efficient-fp16 \
    --remove-bpe=subword_nmt > en-fr.decode.log

2023-02-23 15:58:28 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': 

- Top-K-Sampling

In [26]:
# Generate translations
!fairseq-generate data-bin  \
    --task translation \
    --source-lang en \
    --target-lang fr \
    --path wmt14.en-fr.joined-dict.transformer/model.pt \
    --sampling \
    --sampling-topk 5 \
    --nbest 5 \
    --batch-size 256 \
    --memory-efficient-fp16 \
    --remove-bpe=subword_nmt > en-fr.decode.log

2023-02-23 16:21:37 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': 

# Evaluation

In [27]:
# Extract the hypotheses and references from the decoding log file
!grep ^H en-fr.decode.log | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp.en-fr.txt
!grep ^T en-fr.decode.log | sed 's/^T-//g' | cut -f 2 | sed 's/ @@//g' > ./ref.en-fr.txt

!head ./hyp.en-fr.txt
print("..........")
!head ./ref.en-fr.txt

J&apos; ai voulu faire de ces &quot; ministers &quot; un allié de la communauté de la LGBTQ .
J&apos; ai voulu faire de ces &quot; ministers &quot; des alliés de la communauté du LGBTQ .
Je voulais ouvrir ces postes pour être un allié de la collectivité des LGBTQ .
Je tenais à faire de ces &quot; Adieux &quot; des alliés du LGBTQ .
Je voulais ouvrir ces &quot; BACQ &quot; comme allié de la communauté .
Au début de mon séjour , en 2000 , j&apos; étais très intéressé par les collectivités .
Au début de mon séjour , en 2000 , j&apos; ai manifesté un grand intérêt pour les collectivités .
Au début de mon séjour là @-@ bas en 2001 , j&apos; étais très intéressé aux collectivités .
Au début de mon séjour là @-@ bas , en 2000 , la vie communautaire m&apos; a beaucoup intéressé .
Au début du séjour que j&apos; ai effectué dans cette ville en 2000 , j&apos; étais très intéressée aux communautés .
..........
Je <<unk>> ouvrir ces portes en tant qu&apos; alliée de la communauté LGBTQ .
À mes débu

In [29]:
# Extract only the first hypothesis
!awk 'NR % 5 == 1' ./hyp.en-fr.txt > ./hyp.en-fr.best.txt

In [30]:
# Detokenize text        

with open('./hyp.en-fr.best.txt', encoding='utf8') as fin, open('hyp_detok.en-fr.txt','w', encoding='utf8') as fout:
    for line in fin:
        tokens = md_en.detokenize(line.split(), return_str=True)
        print(tokens, end='\n', file=fout)
        
with open('ref.en-fr.txt', encoding='utf8') as fin, open('ref_detok.en-fr.txt','w', encoding='utf8') as fout:
    for line in fin:
        tokens = md_fr.detokenize(line.split(), return_str=True)
        print(tokens, end='\n', file=fout)

print('Finished detokenizing.')

Finished detokenizing.


In [31]:
!head ./hyp_detok.en-fr.txt
print("..........")
!head ./ref_detok.en-fr.txt

J' ai voulu faire de ces "ministers" un allié de la communauté de la LGBTQ.
Au début de mon séjour, en 2000, j' étais très intéressé par les collectivités.
Après ça, j' étais un peu impatiente de me rendre à la prochaine rendez-vous zéros.
Je suis allé à la maison de Σ pour célébrer - pour la congratuer.
Et je trouvais que c "était super cool, alors je l' ai montré à mon ami.
Je m' appelle Dan, je suis partenaire d' une conquête globale de création.
Je suis retourné à mon laboratoire à Boston et y ai réalisé une petite expérience.
Et le juge l' a reconnu comme un adulte, mais je vois ça.
Nous avons donc là une jeune femme et un jeune homme qui s' adonnent à un jeu d' aumônerie.
J' ai eu le grand privilège, quand j' étais un jeune avocat, de rencontrer "Waters Parks".
..........
Je <<unk>> ouvrir ces portes en tant qu'alliée de la communauté LGBTQ.
À mes débuts en 2000, j'étais très intéressé par les communautés.
Après lui, j'étais un peu <<unk>> pour mon prochain <<unk>> zéro.
Je suis 

- BLEU score (sacreBleu works on detokenized input)

In [32]:
# Evaluate the model
# BLEU score of 18.7 (beam=5), BLEU score of 18.2 (beam=1)
# BLEU score of 15.9 (sampling-topk=5)
!cat ./hyp_detok.en-fr.txt | sacrebleu ./ref_detok.en-fr.txt

{
 "name": "BLEU",
 "score": 15.9,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.3.1",
 "verbose_score": "48.1/24.4/14.1/8.3 (BP = 0.829 ratio = 0.842 hyp_len = 35701 ref_len = 42400)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.3.1"
}


In [33]:
!fairseq-score \
    --sacrebleu \
    --sys ./hyp_detok.en-fr.txt \
    --ref ./ref_detok.en-fr.txt

Namespace(ignore_case=False, order=4, ref='./ref_detok.en-fr.txt', sacrebleu=True, sentence_bleu=False, sys='./hyp_detok.en-fr.txt')
Traceback (most recent call last):
  File "/home/vzhekova/miniconda3/envs/nmt/bin/fairseq-score", line 8, in <module>
    sys.exit(cli_main())
  File "/home/vzhekova/fairseq/fairseq_cli/score.py", line 98, in cli_main
    score(f)
  File "/home/vzhekova/fairseq/fairseq_cli/score.py", line 61, in score
    print(sacrebleu.corpus_bleu(fdsys, [fdref]).format())
  File "/home/vzhekova/miniconda3/envs/nmt/lib/python3.8/site-packages/sacrebleu/compat.py", line 37, in corpus_bleu
    return metric.corpus_score(hypotheses, references)
  File "/home/vzhekova/miniconda3/envs/nmt/lib/python3.8/site-packages/sacrebleu/metrics/base.py", line 414, in corpus_score
    self._check_corpus_score_args(hypotheses, references)
  File "/home/vzhekova/miniconda3/envs/nmt/lib/python3.8/site-packages/sacrebleu/metrics/base.py", line 258, in _check_corpus_score_args
    raise Type

- Gender accuracy

In [18]:
# Evaluate gender accuracy; 0.46
!python /export/data4/vzhekova/MuST-SHE_v1.2/MuST-SHE-v1.1-eval-script/mustshe_acc_v1.1.py \
    --input hyp.en-fr.txt \
    --tsv-definition /export/data4/vzhekova/MuST-SHE_v1.2/MuST-SHE-v1.2-data/tsv/MONOLINGUAL.fr_v1.2.tsv

Category	Term Coverage	Gender Accuracy
-------------------------------------------------
1F	0.03773584905660377	0.16666666666666666
1M	0.07804878048780488	0.5777777777777777
2F	0.12447257383966245	0.3787878787878788
2M	0.11650485436893204	0.5555555555555556
3F	0.0	0.0
3M	0.0	0.0
4F	0.07407407407407407	0.0
4M	0.16666666666666666	0.5
-------------------------------------------------
Global	0.09286463798530954	0.4647887323943662
-------------------------------------------------


# Translation with fine-tuned model

In [5]:
%cd $PATH

/export/data4/vzhekova/biases-data/En-Fr


In [6]:
# Generate translations
!fairseq-generate data-bin  \
    --task translation \
    --source-lang en \
    --target-lang fr \
    --path /export/data4/vzhekova/biases-data/En-Fr_MuST-C/checkpoints/checkpoint_best.pt \
    --beam 5 \
    --batch-size 256 \
    --memory-efficient-fp16 \
    --remove-bpe=subword_nmt > en-fr.decode_finetuned.log

2023-03-10 14:57:29 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': 

In [7]:
# Extract the hypotheses and references from the decoding log file
!grep ^H en-fr.decode_finetuned.log | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp_finetuned.txt
!grep ^T en-fr.decode_finetuned.log | sed 's/^T-//g' | cut -f 2 | sed 's/ @@//g' > ./ref_finetuned.txt

!head ./hyp_finetuned.txt
print("..........")
!head ./ref_finetuned.txt

Je <unk> ouvrir ces <unk> en tant qu <unk> de la communauté LGBQT .
Au début de mon séjour ici en 2000 , j <unk> été très intéressé par les communautés .
Ensuite , j <unk> un peu <unk> d <unk> à mon prochain <unk> o .
Je suis <unk> à la maison de <unk> pour la célébrer - la congrater .
Et je <unk> que c <unk> super cool , alors je le <unk> à mon ami .
Je m <unk> Dan , je suis partenaire d &apos;une association mondiale de création .
Je suis <unk> dans mon laboratoire à Boston et j <unk> fait une petite expérience .
Et le juge l &apos;a certifié comme adulte , mais je vois cette <unk> .
Ici , nous avons une jeune <unk> et un <unk> engagés dans un jeu de <unk> .
J <unk> eu le grand privilège , quand j <unk> jeune avocat , de rencontrer <unk> Parks .
..........
Je <<unk>> ouvrir ces portes en tant qu&apos; alliée de la communauté LGBTQ .
À mes débuts en 2000 , j&apos; étais très intéressé par les communautés .
Après lui , j&apos; étais un peu <<unk>> pour mon prochain <<unk>> zéro .
Je su

In [10]:
# Detokenize text        

md_en = MosesDetokenizer(lang='en')
md_fr = MosesDetokenizer(lang='fr')

with open('hyp_finetuned.txt', encoding='utf8') as fin, open('hyp_finetuned_detok.txt','w', encoding='utf8') as fout:
    for line in fin:
        tokens = md_en.detokenize(line.split(), return_str=True)
        print(tokens, end='\n', file=fout)
        
with open('ref_finetuned.txt', encoding='utf8') as fin, open('ref_finetuned_detok.txt','w', encoding='utf8') as fout:
    for line in fin:
        tokens = md_fr.detokenize(line.split(), return_str=True)
        print(tokens, end='\n', file=fout)

print('Finished detokenizing.')

Finished detokenizing.


In [11]:
# Evaluate the model
# BLEU score of 27.0 (beam=5)
!cat ./hyp_finetuned_detok.txt | sacrebleu ./ref_finetuned_detok.txt

{
 "name": "BLEU",
 "score": 27.0,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.3.1",
 "verbose_score": "61.4/37.1/21.9/10.7 (BP = 1.000 ratio = 1.066 hyp_len = 45219 ref_len = 42400)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.3.1"
}


In [12]:
# Evaluate gender accuracy; 0.49
!python /export/data4/vzhekova/MuST-SHE_v1.2/MuST-SHE-v1.1-eval-script/mustshe_acc_v1.1.py \
    --input hyp_finetuned_detok.txt \
    --tsv-definition /export/data4/vzhekova/MuST-SHE_v1.2/MuST-SHE-v1.2-data/tsv/MONOLINGUAL.fr_v1.2.tsv

Category	Term Coverage	Gender Accuracy
-------------------------------------------------
1F	0.030660377358490566	0.375
1M	0.06829268292682927	0.6571428571428571
2F	0.10548523206751055	0.39285714285714285
2M	0.11262135922330097	0.5285714285714286
3F	0.0	0.0
3M	0.0	0.0
4F	0.07407407407407407	0.0
4M	0.16666666666666666	0.5555555555555556
-------------------------------------------------
Global	0.08342077649527807	0.4946808510638298
-------------------------------------------------
